# <strong>Fashion Outfit Recommendation System</strong>

This notebook implements a system to recommend outfits based on a user's skin tone and preferences. The system involves the following steps:   
1. **Dataset Cleaning and Processing**
2. **Skin Tone Classification from Image**
3. **Using Skin Tone to Decide Colors**
4. **Generating and Displaying Outfit Combinations**
---

## 1. Dataset Cleaning and Processing
In this process, we would like to clean the dataset and prepare it for filtering based on input, such as gender, season, usage, and color.

In [1]:
import pandas as pd 
import numpy as np
import random

#Load the dataset
print ("Dataset Overview:")
data = pd.read_csv(r"C:\EID\fashion-recommender\fashion-dataset\styles.csv", on_bad_lines="skip")
data.sample(10)

Dataset Overview:


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
4370,52840,Women,Accessories,Jewellery,Earrings,Red,Summer,2012.0,Casual,Adrika Red Earrings
27628,29778,Men,Apparel,Bottomwear,Trousers,Brown,Summer,2013.0,Casual,Basics Men Brown Trousers
18333,16980,Men,Footwear,Shoes,Sports Shoes,White,Fall,2011.0,Sports,Puma Men Cell Pavo NM White Sports Shoe
34038,53695,Women,Accessories,Bags,Handbags,Beige,Summer,2012.0,Casual,Kiara Women Beige Handbag
31673,48258,Boys,Apparel,Topwear,Tshirts,Black,Summer,2012.0,Casual,The Amazing Spiderman Boys Black T-Shirt
16298,25887,Unisex,Accessories,Bags,Backpacks,Brown,Winter,2015.0,Travel,American Tourister Unisex Brown Wanderer Backpack
38050,30734,Men,Apparel,Bottomwear,Churidar,White,Summer,2012.0,Ethnic,Fabindia Men White Churidar
24710,17629,Men,Apparel,Topwear,Jackets,Blue,Fall,2011.0,Casual,ADIDAS Men Solid Blue Jacket
41539,22782,Women,Apparel,Topwear,Kurtas,Green,Summer,2011.0,Ethnic,BIBA Women Printed Green Kurta
9677,53676,Women,Accessories,Bags,Handbags,Black,Summer,2012.0,Casual,Kiara Women Black Handbag


In [2]:
#Drop the unnecessary columns
data.drop(["year", "productDisplayName"], axis=1, inplace=True)

In [ ]:
#Checking the missing values inside the dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44424 entries, 0 to 44423
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              44424 non-null  int64 
 1   gender          44424 non-null  object
 2   masterCategory  44424 non-null  object
 3   subCategory     44424 non-null  object
 4   articleType     44424 non-null  object
 5   baseColour      44409 non-null  object
 6   season          44403 non-null  object
 7   usage           44107 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.7+ MB
